# Recommend Movielens

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/project/AIFFEL_Study/e7_recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'counts'}, inplace=True)

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/project/AIFFEL_Study/e7_recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 데이터 분석
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기있는 영화 30개(인기순)

In [5]:
ratings = pd.merge(ratings,movies)
ratings.head(10)

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
5,18,1193,4,978156168,One Flew Over the Cuckoo's Nest (1975),Drama
6,19,1193,5,982730936,One Flew Over the Cuckoo's Nest (1975),Drama
7,24,1193,5,978136709,One Flew Over the Cuckoo's Nest (1975),Drama
8,28,1193,3,978125194,One Flew Over the Cuckoo's Nest (1975),Drama
9,33,1193,5,978557765,One Flew Over the Cuckoo's Nest (1975),Drama


In [6]:
using_cols = ['user_id','title','counts']
ratings = ratings[using_cols]
ratings.head()

,user_id,title,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5


In [7]:
# ratings에 있는 유니크한 영화 개수
ratings['title'].nunique()

3628

In [8]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [9]:
# 인기 있는 영화 30개(인기순)
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [10]:
ratings.head()

,user_id,title,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5


In [11]:
# 유저별 몇 개의 영화를 보고 있는지에 대한 통계
user_count = ratings.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

## 내가 선호하는 영화 5가지 골라서 Data에 추가

In [12]:
my_favorite = ['Men in Black (1997)' , 'Toy Story (1995)' ,'Matrix, The (1999)' 
               ,'Alien (1979)' ,'Gladiator (2000)']

# 'hyeon'이라는 user_id가 위 영화를 30회씩 봤다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['hyeon']*5, 'title': my_favorite, 'counts':[30]*5})

if not ratings.isin({'user_id':['hyeon']})['user_id'].any():  # user_id에 'hyeon'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,counts
836473,5851,One Little Indian (1973),5
836474,5854,Slaughterhouse (1987),4
836475,5854,"Promise, The (Versprechen, Das) (1994)",3
836476,5938,"Five Wives, Three Secretaries and Me (1998)",4
836477,5948,Identification of a Woman (Identificazione di ...,5
0,hyeon,Men in Black (1997),30
1,hyeon,Toy Story (1995),30
2,hyeon,"Matrix, The (1999)",30
3,hyeon,Alien (1979),30
4,hyeon,Gladiator (2000),30


In [14]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
title_unique = ratings['title'].unique()

# 유저, 타이틀 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [15]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['hyeon'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(title_to_idx['Men in Black (1997)'])

6039
175


In [16]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = ratings['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,175,30
1,6039,40,30
2,6039,124,30
3,6039,193,30


In [17]:
data = ratings
data

,user_id,title,counts
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,175,30
1,6039,40,30
2,6039,124,30
3,6039,193,30


In [18]:
# 1회만 count한 데이터의 비율을 보는 코드
only_one = data[data['counts']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

0,836483
Ratio of only_one over all data is 0.00%


In [19]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [20]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [21]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [22]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [23]:
# 모델 훈련
als_model.fit(csr_data_transpose)

In [24]:
hyeon, Men_in_Black = user_to_idx['hyeon'], title_to_idx['Men in Black (1997)']
hyeon_vector, Men_in_Black_vector = als_model.user_factors[hyeon], als_model.item_factors[Men_in_Black]

In [25]:
hyeon_vector

array([ 0.46470326,  0.01044457,  0.394218  , -0.75427365, -0.42003673,
        0.9674789 , -0.840361  , -1.157182  , -0.04243235, -1.5557613 ,
        0.17428333,  0.3428291 , -2.031921  ,  0.9586978 , -0.24166636,
        1.7045459 ,  1.6104146 , -0.15051587, -0.74611986,  0.37736493,
       -0.9097534 ,  1.0733541 , -0.5639817 , -1.2811619 ,  0.01352711,
        2.2173839 ,  1.8365786 , -0.50095487,  0.6339386 ,  0.46081054,
        0.24739377, -0.48473912, -0.39699876, -0.71369624,  0.9058312 ,
        1.7865157 , -0.28821808,  0.2454271 ,  1.2352034 , -1.2064062 ,
        0.45602503, -1.0497007 , -1.3420143 ,  0.07337721,  0.11259513,
       -0.7528897 , -0.25240457,  1.2968744 , -0.92051   ,  0.58881557,
       -0.07954125,  0.13135791, -0.79813135,  0.03049586, -0.90085506,
       -0.42976043,  1.2404104 , -0.0601238 , -0.7706534 ,  0.3471684 ,
        0.778938  , -0.5812121 , -0.7653607 , -1.1824902 ,  0.44571623,
       -0.02395608, -0.13564534,  0.5366308 ,  0.5164422 ,  1.24

In [26]:
Men_in_Black_vector

array([ 2.92837135e-02,  1.23353377e-02, -1.76368672e-02,  1.21374126e-03,
        2.87794787e-02,  5.06036542e-02,  2.06431635e-02, -9.30931710e-04,
       -2.09621224e-03, -2.86945561e-03, -4.58484190e-03,  2.74477769e-02,
        5.94754005e-03,  7.97022413e-03,  5.12850694e-02,  2.39209328e-02,
        9.22299176e-03,  2.61546802e-02, -1.92229841e-02,  1.74798612e-02,
       -3.01835593e-03,  3.50111239e-02, -2.44328137e-02, -1.81706343e-02,
        1.74293872e-02,  1.69168245e-02,  3.16688605e-02, -4.64050146e-03,
       -3.93338595e-03, -8.96057766e-03,  2.35689711e-02,  2.54392158e-03,
        3.55045637e-03, -5.07078925e-03,  2.04439685e-02,  1.99066531e-02,
       -2.35491917e-02,  3.72108407e-02,  2.51118317e-02, -1.62993150e-03,
        5.52983220e-05, -2.60474086e-02, -1.65579822e-02,  1.38679352e-02,
        1.24575533e-02, -2.29964871e-02,  1.67823285e-02, -1.12207178e-02,
       -1.59457438e-02,  9.29933134e-03, -2.30531339e-02, -6.34295808e-04,
        3.00862473e-02, -

In [27]:
# hyeon과 Men_in_Black를 내적하는 코드
np.dot(hyeon_vector, Men_in_Black_vector)

0.8335755

In [28]:
Alien = title_to_idx['Alien (1979)']
Alien_vector = als_model.item_factors[Alien]
np.dot(hyeon_vector, Alien_vector)

0.84569037

In [29]:
favorite_Movie = 'Matrix, The (1999)' 
movie_id = title_to_idx[favorite_Movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(124, 0.17508712),
 (92, 0.13384263),
 (62, 0.11839689),
 (141, 0.104730636),
 (200, 0.10408816),
 (145, 0.09651522),
 (375, 0.09587555),
 (107, 0.08365051),
 (75, 0.08314275),
 (175, 0.079353005),
 (317, 0.076721355),
 (44, 0.073264934),
 (117, 0.06947825),
 (193, 0.06638111),
 (651, 0.065074034)]

In [30]:
#title_to_idx 를 뒤집어, index로부터 title 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Terminator, The (1984)',
 'Fifth Element, The (1997)',
 'Face/Off (1997)',
 'Jurassic Park (1993)',
 'Hunt for Red October, The (1990)',
 'Men in Black (1997)',
 'Twelve Monkeys (1995)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Alien (1979)',
 'Aliens (1986)']

In [31]:
def get_similar_movie(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title


In [32]:
get_similar_movie('Gladiator (2000)')

['Gladiator (2000)',
 'Mission: Impossible 2 (2000)',
 'Patriot, The (2000)',
 'Perfect Storm, The (2000)',
 'X-Men (2000)',
 'U-571 (2000)',
 'Frequency (2000)',
 'Gone in 60 Seconds (2000)',
 'Shanghai Noon (2000)',
 'Machine, The (1994)']

In [33]:
get_similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Groundhog Day (1993)',
 'Babe (1995)',
 'Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 'Pleasantville (1998)',
 "There's Something About Mary (1998)"]

In [34]:
user = user_to_idx['hyeon']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(92, 1.0042546),
 (107, 0.7706559),
 (372, 0.69888055),
 (200, 0.6705761),
 (62, 0.6203221),
 (651, 0.6058663),
 (50, 0.60497284),
 (346, 0.5458844),
 (482, 0.51208466),
 (322, 0.50420606),
 (197, 0.49594763),
 (44, 0.48105913),
 (87, 0.45548946),
 (474, 0.44260773),
 (22, 0.41344678),
 (145, 0.4080794),
 (150, 0.40613422),
 (317, 0.399357),
 (126, 0.3899256),
 (602, 0.38313523)]

In [35]:
[idx_to_title[i[0]] for i in movie_recommended]

['Terminator 2: Judgment Day (1991)',
 'Jurassic Park (1993)',
 'X-Men (2000)',
 'Terminator, The (1984)',
 'Total Recall (1990)',
 'Aliens (1986)',
 'Toy Story 2 (1999)',
 'Patriot, The (2000)',
 'Mission: Impossible 2 (2000)',
 'Babe (1995)',
 'Jaws (1975)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Braveheart (1995)',
 'Chicken Run (2000)',
 'Back to the Future (1985)',
 'Fifth Element, The (1997)',
 'Independence Day (ID4) (1996)',
 'Twelve Monkeys (1995)',
 'Shakespeare in Love (1998)',
 '2001: A Space Odyssey (1968)']

In [36]:
Chicken_Run = title_to_idx['Chicken Run (2000)']
explain = als_model.explain(user, csr_data, itemid=Chicken_Run)

In [37]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Gladiator (2000)', 0.2513064450035536),
 ('Toy Story (1995)', 0.17853205016381174),
 ('Men in Black (1997)', 0.11094021749074086),
 ('Alien (1979)', 0.030882810565093788),
 ('Matrix, The (1999)', -0.1328483377895596)]

In [38]:
Terminator_2 = title_to_idx['Terminator 2: Judgment Day (1991)']
explain = als_model.explain(user, csr_data, itemid=Terminator_2)

In [39]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Matrix, The (1999)', 0.4829755280448201),
 ('Men in Black (1997)', 0.3371204149026675),
 ('Alien (1979)', 0.18671953853032966),
 ('Toy Story (1995)', 0.018565737621070423),
 ('Gladiator (2000)', -0.043005920257099116)]

## 마치며
생각보다 추천시스템이 엄청 복잡하다고 느꼈다, 그렇게 많은 데이터를 넣어 준 것도 아닌데 신기하게 내가 재밌게 봤던 영화들을 추천하는 것을 보고 엄청 정교하다고 느꼈다 좀더 정교한 데이터와 사용자의 여러 행동들을 잘 처리하여 입력한다면 좋은 추천 시스템 모델이 나올 수 있다고 생각한다.